In [1]:
#pip install chess
#!pip install pgn2data
from converter.pgn_data import PGNData
import chess.pgn
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import re
import numpy as np

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
C:\Users\79160\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Первичное преобразование файла

In [ ]:
#Перевод файла формата .pgn в .csv датасет
pgn_data = PGNData("lichess_db_standard_rated_2014-01.pgn")
result = pgn_data.export()
#в результате данная библиотека выводит два файла:
if result.is_complete:
    #файл с основным описанием игры
    games_df = result1.get_games_df()
    print(games_df.head())
    #файл с ходами, сделанными в каждой игре, и некоторыми дополнительными характеристиками
    moves_df = result1.get_moves_df()
    print(moves_df.head())

На выходе получаем два .csv файла: **'lichess_db_standard_rated_2013-08_moves.csv'** - это moves_df, и **'lichess_db_standard_rated_2013-08_game_info.csv'** - это games_df

### Дополнительные характеристики

1. **Оценки ходов**

In [ ]:
#создадим функцию для более удобного чтения исходных файлов игры
def read_pgn_file(file_path):
    with open(file_path) as f:
        game = chess.pgn.read_game(f)
        while game is not None:
            yield game
            game = chess.pgn.read_game(f)

In [ ]:
#Выведем некоторые признаки:
c1_list = [] #порядковый номер игры
c2_list = [] #ход в игре
c3_list = [] # оценка хода

for i, game in tqdm(enumerate(read_pgn_file('lichess_db_standard_rated_2013-08.pgn'))):
    numb = i + 1
    node = game
    while not node.is_end():
        next_node = node.variation(0)
        move = node.board().san(next_node.move)
        evaluation = next_node.eval()
        c1_list.append(numb)
        c2_list.append(move)
        c3_list.append(evaluation)
        node = next_node

In [ ]:
# Далее преобразуем листы в np.array и создадим соответсвующий датафрейм
c1 = np.array(c1_list)
c2 = np.array(c2_list)
c3 = np.array(c3_list)

new_data = pd.DataFrame()
new_data['Game'] = c1
new_data['Move'] = c2
new_data['Eval'] = c3

# И запишем результат в отдельный файл
new_data.to_csv('evals.csv')

1.1. Дополнительное преобразование оценок ходов

In [ ]:
epic = new_data[new_data['Eval'].notnull()]
epic.loc[:, ['Eval']] = epic.loc[:, ['Eval']].astype(str) #Оценки имеют игровой формат PovScore

In [ ]:
# Получим список чисел,соответствующих оценку хода
numbers = []
strings = upd_moves_df['eval'].values
for string in strings:
    matches = re.findall(r'-?\d+', string)
    for match in matches:
        numbers.append(int(match))
len(numbers), upd_moves_df['eval'].shape

2. **Дебюты**

In [ ]:
# Извлечем из исходого датасета все дебюты, используя уже имеющуюся функцию чтения игр .pgn
openings = []
for i, game in tqdm(enumerate(read_pgn_file('lichess_db_standard_rated_2013-08.pgn'))):
        headers = game.headers
        for key, value in headers.items():
            if key == 'Opening':
                openings.append(value)

In [ ]:
op = np.array(openings)

### Формирование финальных датасетов

Так как основным призаком, используещимся в данном проекте является оценка ходов игроков, в соотвествии с наличием подобных данных необходимо сократить имеющееся количество игр в датасете.

In [3]:
# Добавим в датасат с ходами полученные численные оценки
moves_df['eval'] = new_data['Eval']
# Оставим только те, для которых есть эти оценки
upd_moves_df = moves_df[moves_df['eval'].notnull()]
#Убедимся, что значения оценок, которые находились в формате PovScore превратятся в строки
upd_moves_df.loc[:, ['Eval']] = upd_moves_df.loc[:, ['Eval']].astype(str) #Оценки имеют игровой формат PovScore

* Дополнительное преобразование оценок ходов

In [ ]:
# Получим список чисел,соответствующих оценку хода
numbers = []
strings = upd_moves_df['eval'].values
for string in strings:
    matches = re.findall(r'-?\d+', string)
    for match in matches:
        numbers.append(int(match))
len(numbers), upd_moves_df['eval'].shape

In [ ]:
# Обновим значение колонки
upd_moves_df['eval'] = numbers

### Теперь преобразуем датасет с общим описанием игр

In [ ]:
#Добавим дебюты
games_df['op'] = op
# Оставшиеся игры
games_left = upd_moves_df['game_id'].values
upd_games_df = games_df[games_df['game_id'].isin(games_left)]

In [ ]:
# Оставим только интерпретируемые и некопирующиеся элементы
upd_games_df = upd_games_df[['game_id', 'event', 'white', 'black', 'result', 'white_elo',
       'black_elo', 'winner_loser_elo_diff', 'eco', 'termination', 'time_control', 'op']]

In [ ]:
#Запишем финальные датасеты в .csv файлы
upd_moves_df.to_csv('upd_moves_df.csv')
upd_games_df.to_csv('upd_games_df.csv')